# Checking URLs in excel with python

### Usage

* Upload your xlsx file in the **files** section on the left.
* Right-click the file name and select **Copy path to clipboard**
* In the code block below, in **line 35**, replace the `file_path` with the path to your excel file
* Click the **Run notebook** button above to execute the code. The results will be displayed in the output area below the final code block.

In [ ]:
# install required modules
!pip install openpyxl

In [ ]:
# load the required modules
import openpyxl
import re
import requests
import warnings

### Notes
We need to iterate through each sheet, and find anything that looks like a URL. 

Our assumptions:
* a string is a URL if and only if it starts with http or https
* any strings that start with http or https but contain ; or ' ' should be split into multiple strings
* any URLs that end in a . or a ; should have that stripped

Then, we need to attempt to connect to each URL with a GET request and record whether it returns a success response or a failure.
We only want to check a URL once- the script should skip any that have already been checked.

Finally, we only want to return a list of the URLs reporting an error

### Code

In [ ]:
def find_urls_in_excel(file_path):
    # ignore warnings from openpyxl, we're not writing to the excel file
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
    # Load the workbook
    workbook = openpyxl.load_workbook(file_path)
    
    url_pattern = re.compile(r'(https?://[^\s]+)')
    
    url_list = []

    # Iterate through each sheet
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        # Iterate through each row and column
        for row in sheet.iter_rows():
            for cell in row:
                if cell.value:
                    urls = url_pattern.findall(str(cell.value))
                    for url in urls:
                        sanitized_url = url.rstrip('.,!?;:)')
                        url_list.append({'sheet': sheet_name, 'cell': cell.coordinate, 'url': sanitized_url})
    return url_list

# check the status of each URL
def check_url_status(url):
    try:
        response = requests.get(url)
        return response.status_code
    except requests.exceptions.RequestException as e:
        # print(f"Error: An exception occurred while trying to reach the URL '{url}'. Exception details: {e}")
        return "Unspecified Error"
    

# put the path to the excel file here
file_path = '/work/tanzania_2019_2020.xlsx'
urls = find_urls_in_excel(file_path)
checked_urls = []
for entry in urls:
    if entry['url'] not in checked_urls:
        status_code = check_url_status(entry['url'])
        checked_urls.append(entry['url'])  # Mark this URL as checked
        if status_code != 200:
            print(f"Sheet: {entry['sheet']}, Cell: {entry['cell']}, URL: {entry['url']}, Status: {status_code}")
    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0a5edc1a-b9e6-43d0-b3b1-b9ae11781243' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>